# Analysis of Toronto neighborhoods using Foursquare API

by Tom Fenske

In [76]:
#Importing necessary libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import folium
import json 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported!')

Libraries imported!


In [79]:
#Reading the .csv with the Toronto neighborhoods

file_path='../../II_Datasets/toronto_w_latlon.csv'#going two folders up and then enter folder "II_Datasets"
df_neigh=pd.read_csv(file_path)
df_neigh.drop('Unnamed: 0', axis=1, inplace=True)
df_neigh.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [80]:
df_neigh.shape

(99, 5)

In [86]:
CLIENT_ID = 'LEU5OIAP31MI5JXX1M4LGWJNMMPHZWTKHQD3W5A34FGSDPY0' # your Foursquare ID
CLIENT_SECRET = '0EKHGNCNUD014O4USGZ03ZDFXQQEDNEHOMP0OJQSGBIDAMEY' # your Foursquare Secret
VERSION = '20200719' # Foursquare API version
radius=1500
LIMIT=50

In [87]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
toronto_venues = getNearbyVenues(names=df_neigh['Neighbourhood'],
                                   latitudes=df_neigh['Latitude'],
                                   longitudes=df_neigh['Longitude']
                                  )



Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
B

In [89]:
print(toronto_venues.shape)
toronto_venues.head()
toronto_venues['Neighbourhood'].nunique()

(4325, 7)


99

In [90]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 298 uniques categories.


In [91]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,ATM,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store

In [92]:
toronto_onehot.shape

(4325, 299)

In [93]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,ATM,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store

In [94]:
toronto_grouped.shape

(99, 299)

In [95]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [96]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Bakery,Breakfast Spot,Coffee Shop,Gym / Fitness Center,Caribbean Restaurant,Shopping Mall,Asian Restaurant,Sri Lankan Restaurant,Korean Restaurant
1,"Alderwood, Long Branch",Park,Coffee Shop,Burger Joint,Toy / Game Store,Café,Bank,Grocery Store,Pizza Place,Discount Store,Burrito Place
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Pizza Place,Coffee Shop,Gas Station,Bank,Sandwich Place,Restaurant,Fried Chicken Joint,Sushi Restaurant,French Restaurant
3,Bayview Village,Bank,Park,Gas Station,Trail,Café,Chinese Restaurant,Grocery Store,Shopping Mall,Dog Run,Japanese Restaurant
4,"Bedford Park, Lawrence Manor East",Bagel Shop,Sushi Restaurant,Bakery,Italian Restaurant,Pub,Coffee Shop,Fast Food Restaurant,Café,Asian Restaurant,Japanese Restaurant


Cluster Neighbourhoods

In [98]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:200] 

array([2, 3, 2, 3, 3, 1, 1, 1, 3, 3, 3, 2, 2, 1, 3, 1, 2, 2, 1, 3, 3, 2,
       3, 2, 1, 3, 3, 2, 2, 1, 3, 1, 2, 2, 2, 1, 3, 2, 3, 2, 3, 3, 2, 2,
       3, 2, 1, 3, 2, 3, 0, 2, 2, 3, 3, 2, 3, 1, 2, 3, 3, 2, 2, 1, 1, 1,
       3, 3, 3, 3, 2, 2, 2, 1, 3, 2, 1, 3, 3, 1, 3, 3, 2, 2, 1, 3, 4, 2,
       1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2])

In [101]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.dropna()
toronto_merged = df_neigh

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Coffee Shop,Pharmacy,Convenience Store,Supermarket,Intersection,Bank,Gas Station,Discount Store,Sandwich Place,Caribbean Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Middle Eastern Restaurant,Gym,Fast Food Restaurant,Grocery Store,Shoe Store,Wings Joint,Portuguese Restaurant,Hockey Arena,Supermarket
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Restaurant,Farmers Market,Pub,Diner,Bakery,Performing Arts Venue,Italian Restaurant,Park,Mediterranean Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Greek Restaurant,Cosmetics Shop,Dessert Shop,Restaurant,Fried Chicken Joint,Bookstore,Sushi Restaurant,Frozen Yogurt Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Pizza Place,Park,Gastropub,Restaurant,Middle Eastern Restaurant,Jewelry Store,Sushi Restaurant,Juice Bar,Supermarket


In [103]:
# create map
toronto_lat=43.651070
toronto_lon=-79.347015

map_clusters = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Restaurant,Coffee Shop,Caribbean Restaurant,Bank,Gas Station,Financial or Legal Service,Chinese Restaurant


In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Restaurant,Farmers Market,Pub,Diner,Bakery,Performing Arts Venue,Italian Restaurant,Park,Mediterranean Restaurant
3,North York,1,Clothing Store,Furniture / Home Store,Greek Restaurant,Cosmetics Shop,Dessert Shop,Restaurant,Fried Chicken Joint,Bookstore,Sushi Restaurant,Frozen Yogurt Shop
4,Downtown Toronto,1,Coffee Shop,Pizza Place,Park,Gastropub,Restaurant,Middle Eastern Restaurant,Jewelry Store,Sushi Restaurant,Juice Bar,Supermarket
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Theater,Pizza Place,Plaza,Gastropub,Shopping Mall,Seafood Restaurant
11,Etobicoke,1,Park,Hotel,Restaurant,Farmers Market,Electronics Store,Café,Bank,Theater,Sandwich Place,Grocery Store
14,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Seafood Restaurant,Italian Restaurant,American Restaurant,Farmers Market,Clothing Store,Theater,Gym
19,Downtown Toronto,1,Coffee Shop,Restaurant,Japanese Restaurant,Hotel,Seafood Restaurant,Café,Beer Bar,Gym,Farmers Market,Beach
23,Downtown Toronto,1,Coffee Shop,Park,Gastropub,Burrito Place,Theater,Clothing Store,Plaza,Yoga Studio,Pizza Place,Department Store
29,Downtown Toronto,1,Hotel,Theater,Coffee Shop,Clothing Store,Café,Gym,Beer Bar,Plaza,Gastropub,Speakeasy
35,Downtown Toronto,1,Hotel,Coffee Shop,Park,Brewery,Plaza,Café,Baseball Stadium,Chinese Restaurant,Scenic Lookout,Beer Bar


In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Coffee Shop,Pharmacy,Convenience Store,Supermarket,Intersection,Bank,Gas Station,Discount Store,Sandwich Place,Caribbean Restaurant
1,North York,2,Coffee Shop,Middle Eastern Restaurant,Gym,Fast Food Restaurant,Grocery Store,Shoe Store,Wings Joint,Portuguese Restaurant,Hockey Arena,Supermarket
5,Etobicoke,2,Bank,Shopping Mall,Pharmacy,Bakery,Playground,Liquor Store,Convenience Store,Golf Course,Bus Line,Supermarket
8,East York,2,Pizza Place,Pharmacy,Fast Food Restaurant,Park,Convenience Store,Coffee Shop,Restaurant,Rock Climbing Spot,Sandwich Place,Café
10,North York,2,Coffee Shop,Fast Food Restaurant,Bank,Grocery Store,Caribbean Restaurant,Fried Chicken Joint,Restaurant,Gas Station,Pizza Place,Sandwich Place
13,East York,2,Coffee Shop,Pizza Place,Park,Sandwich Place,Gastropub,Breakfast Spot,Pharmacy,Bank,Thai Restaurant,Ice Cream Shop
16,Etobicoke,2,Baseball Field,Coffee Shop,Pizza Place,Grocery Store,Bank,Liquor Store,Beer Store,Stadium,Mexican Restaurant,Fast Food Restaurant
17,Scarborough,2,Pizza Place,Restaurant,Breakfast Spot,Fast Food Restaurant,Bank,Coffee Shop,Park,Discount Store,Sandwich Place,Juice Bar
21,Scarborough,2,Coffee Shop,Pizza Place,Fast Food Restaurant,Bank,Sandwich Place,Pharmacy,Café,Chinese Restaurant,Bakery,Supermarket
22,East York,2,Coffee Shop,Bakery,Indian Restaurant,Grocery Store,Restaurant,Thai Restaurant,Burger Joint,Sushi Restaurant,Sporting Goods Shop,Curling Ice


In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,North York,3,Bank,Japanese Restaurant,Coffee Shop,Restaurant,Café,Italian Restaurant,Supermarket,Pizza Place,Burger Joint,Middle Eastern Restaurant
12,Scarborough,3,Hotel,Burger Joint,Park,Grocery Store,Gym,Gym / Fitness Center,Neighborhood,Italian Restaurant,Breakfast Spot,Filipino Restaurant
15,York,3,Coffee Shop,Italian Restaurant,Caribbean Restaurant,Indian Restaurant,Café,Bagel Shop,BBQ Joint,Frozen Yogurt Shop,Ice Cream Shop,Middle Eastern Restaurant
18,East Toronto,3,Pub,Breakfast Spot,Beach,Coffee Shop,BBQ Joint,Tea Room,Park,Caribbean Restaurant,Japanese Restaurant,Thai Restaurant
20,York,3,Italian Restaurant,Furniture / Home Store,Bakery,Coffee Shop,Mexican Restaurant,Caribbean Restaurant,Grocery Store,Pharmacy,Sushi Restaurant,Bank
24,Downtown Toronto,3,Café,Coffee Shop,Korean Restaurant,Indian Restaurant,Cocktail Bar,Grocery Store,Restaurant,Diner,Comedy Club,Latin American Restaurant
30,West Toronto,3,Café,Coffee Shop,Italian Restaurant,Bakery,Bar,Brewery,Portuguese Restaurant,Beer Bar,Brazilian Restaurant,Thai Restaurant
34,East York,3,Greek Restaurant,Café,Bakery,Coffee Shop,Pizza Place,American Restaurant,Park,Ethiopian Restaurant,Gastropub,Brewery
36,West Toronto,3,Bakery,Café,Asian Restaurant,Italian Restaurant,Coffee Shop,Cocktail Bar,Pizza Place,Paper / Office Supplies Store,Brewery,Tapas Restaurant
38,North York,3,Bank,Park,Gas Station,Trail,Café,Chinese Restaurant,Grocery Store,Shopping Mall,Dog Run,Japanese Restaurant


In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Scarborough,4,Donut Shop,Farm,National Park,Zoo Exhibit,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
